In [98]:
import sys
#import os
import pickle
import pandas as pd
import numpy as np
from pyarrow import feather
from scipy import sparse
from queue import Queue
from tqdm import tqdm
import itertools
#import concurrent
import multiprocessing
from firebase_helper import *
import math

In [14]:
fb = firebase_helper()
db = fb.get_db()

In [15]:
db

In [30]:
ticker_map_path = "/mnt/d/data/news/ticker_maping_dict.pkl"
ticker_map_dict = pickle.load(open(ticker_map_path, "rb"))

In [ ]:
#overwrite ticker mapping on db
doc_ref = db.collection('ticker_map').document('dict')
doc_ref.set(ticker_map_dict)

In [ ]:
knn_ref = db.collection("knn").document("knn_ticker")

In [ ]:
knn_data = np.load("knn_ticker.npy")

In [2]:
from typing import List
import os
from google.cloud.firestore_v1 import batch
from google.cloud.firestore_v1.batch import WriteBatch
from google.cloud.firestore_v1.client import Client
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

max_cpus = 12  # Maximum number of CPUs to use
batch_size = 500  # Batch size for Firestore writes

def insert_data(batch: batch, i: int) -> None:
    lc = knn_data[i]
    temp_map = {str(j): lc[j] for j in range(len(lc)) if lc[j] != 0}
    batch.set(db.collection('knn_news').document(str(i)), temp_map)

def process_batch(batch: WriteBatch, indices: List[int]) -> None:
    for i in indices:
        insert_data(batch, i)

with ThreadPoolExecutor(max_workers=min(max_cpus, os.cpu_count())) as executor:
    for i in tqdm(range(0, len(knn_data), batch_size)):
        batch = db.batch()
        process_batch(batch, range(i, min(i + batch_size, len(knn_data))))
        batch.commit()


NameError: name 'knn_data' is not defined

In [ ]:
n = len(knn_data)
base_user_pref_vector = np.zeros(n)

In [57]:
def memoize(function):
    cache = {}
    def wrapper(input):
        if input not in cache:
            cache[input] = function(input)
        return cache[input]
    return wrapper

@memoize
def sic_match(input):
    sic_codes = {
        '01': 'agriculture',
        '02': 'agriculture',
        '07': 'agriculture',
        '08': 'agriculture',
        '09': 'agriculture',
        '10': 'mining',
        '11': 'mining',
        '12': 'mining',
        '13': 'mining',
        '14': 'mining',
        '15': 'construction',
        '16': 'construction',
        '17': 'construction',
        **{f"{i:02d}": "manufacturing" for i in range(20, 40)},
        **{f"{i:02d}": "transportation" for i in range(40, 50)},
        '50': 'wholesale',
        '51': 'wholesale',
        **{f"{i:02d}": "retail" for i in range(52, 60)},
        **{f"{i:02d}": "finance" for i in range(60, 68)},
        **{f"{i:02d}": "services" for i in range(70, 90)},
        **{f"{i:02d}": "public_administration" for i in range(91, 100)},
    }
    try:
        return sic_codes[input]
    except KeyError:
        raise ValueError("Invalid input. Please enter a two-character string matching a valid SIC code.")


In [58]:
print(sic_match("01"))

agriculture


In [27]:
#base imports
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
poly_dir = os.path.abspath(os.path.join(os.getcwd(), 'data_poly'))
sys.path.append(poly_dir)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import itertools

# package imports
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from tqdm import tqdm
import pyarrow.feather as feather

# local imports
import data_poly.poly_getdata as poly_getdata
import data_poly.poly_url as poly_url
import data_poly.poly_helper as poly_helper

In [28]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="/home/peterzerg/repos/PeterZergQuant/.ENV")
api_key = os.environ.get("POLYGON_APIKEY_MASTER")

In [59]:
url_factory = poly_url.StockUrlFactory(api_key)
ticker_lc = ticker_map_dict.keys()
urls_dict = {ticker: url_factory.ReferenceData.ticker_info(url_factory, ticker) for ticker in ticker_lc}
df_dict = await poly_helper.get_data_from_urls(urls_dict)
df = pd.concat(df_dict.values())

In [32]:
upsert_dict = {ticker: df.to_dict('records')[0] for ticker, df in df_dict.items()}

upload ticker info to db

In [5]:
def chunks(data, size):
    data_keys = list(data.keys())
    for i in range(0, len(data_keys), size):
        yield {k: data[k] for k in data_keys[i:i+size]}

In [31]:
# Get the reference to the ticker_info collection
collection_ref = db.collection('ticker_info')

# Split upsert_dict into chunks with a maximum size of 500
chunk_size = 500
sub_dicts = list(chunks(upsert_dict, chunk_size))

# Iterate through the sub_dicts
for sub_dict in sub_dicts:
    # Create a batch to batch the writes
    batch = db.batch()
    
    # Iterate through the sub_dict
    for key, value in sub_dict.items():
        # Add the set operation to the batch
        doc_ref = collection_ref.document(key)
        batch.set(doc_ref, value)
    
    # Commit the batch
    batch.commit()

upload ticker sic mapping to db

In [60]:
sic_map_dict = {
    str(ticker): sic_match(sic_code[:2]) if isinstance(sic_code, str) else None
    for ticker, sic_code in zip(df.ticker, df.sic_code)
}

In [61]:
#upload data to db
doc_ref = db.collection('ticker_sic_map').document('dict')
doc_ref.set(sic_map_dict)

update_time {
  seconds: 1679430267
  nanos: 25464000
}

upload news data to db

In [62]:
# Load the data
news_data = pd.read_feather("/mnt/d/data/news/local_us_equity_news")

In [63]:
# Get reference to the document
doc_ref = db.collection('ticker_sic_map').document('dict')

# Retrieve the document data
doc = doc_ref.get()

# Check if the document exists
if doc.exists:
    # Get the document data
    ticker_sic_map = doc.to_dict()
else:
    print(f"No such document: {doc_ref.id}")

In [64]:
#add 10 industry cols
news_data["industries"] = news_data.tickers.apply(lambda tickers: [ticker_sic_map.get(ticker, None) for ticker in tickers])

In [65]:
# Define the industry column names and default values
industry_cols = list(set(['agriculture', 'mining', 'construction', 'manufacturing', 'transportation',
                 'wholesale', 'retail', 'finance', 'services', 'public_administration']))
default_value = False

# Define a function to convert the industries list to a boolean list
def to_boolean_list(industries):
    return [col in industries for col in industry_cols]

# Create a dataframe with the boolean values for each industry
boolean_df = pd.DataFrame(tqdm(news_data['industries'].apply(to_boolean_list).tolist()), columns=industry_cols)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 819163/819163 [00:00<00:00, 6138130.68it/s]


In [68]:
news_data = news_data.reset_index(drop=True)
boolean_df = boolean_df.reset_index(drop=True)
news_data = pd.concat([news_data, boolean_df], axis=1)

In [72]:
# Convert arrays to lists
def convert_arrays_to_lists(value):
    if isinstance(value, (list, np.ndarray)):
        return list(value)
    return value

news_data = news_data.applymap(convert_arrays_to_lists)

# Convert the DataFrame to a dictionary format
news_data_dict = news_data.set_index('id').T.to_dict()

/tmp/ipykernel_817/279113026.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  news_data_dict = news_data.set_index('id').T.to_dict()


In [74]:
# Split the news_data_dict into chunks of 500
chunk_size = 500
sub_dicts = list(chunks(news_data_dict, chunk_size))

# Iterate through the sub_dicts
for sub_dict in tqdm(sub_dicts):
    # Create a batch to batch the writes
    batch = db.batch()

    # Iterate through the key-value pairs in the sub_dict
    for key, value in sub_dict.items():
        # Set the document reference in the news_data collection using the key
        doc_ref = db.collection('news_data').document(str(key))
        
        # Add the key-value pair to the batch
        batch.set(doc_ref, value)
    
    # Commit the batch
    batch.commit()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 617/617 [32:20<00:00,  3.14s/it]


In [75]:
#incase we want to use async
import asyncio
from google.cloud import firestore
from tqdm import tqdm

# Initialize the Firestore client
db = firestore.Client()

# Define a coroutine for writing a document to Firestore
async def write_document(doc_ref, data):
    await doc_ref.set(data)

# Define a function to write a batch of documents to Firestore asynchronously
async def write_batch(batch):
    # Create a semaphore to limit the number of concurrent tasks
    semaphore = asyncio.Semaphore(12)

    # Create a list to hold the coroutines
    coroutines = []

    # Iterate through the batch and create a coroutine for each document
    for key, value in batch.items():
        # Set the document reference in the news_data collection using the key
        doc_ref = db.collection('news_data').document(str(key))

        # Create a coroutine to write the document to Firestore
        coroutine = write_document(doc_ref, value)

        # Append the coroutine to the list
        coroutines.append(coroutine)

    # Run the coroutines concurrently with the semaphore
    async with semaphore:
        await asyncio.gather(*coroutines)

# Split the news_data_dict into chunks of 500
chunk_size = 500
sub_dicts = list(chunks(news_data_dict, chunk_size))

# Iterate through the sub_dicts
for sub_dict in tqdm(sub_dicts):
    # Create a batch to batch the writes
    batch = {}

    # Iterate through the key-value pairs in the sub_dict
    for key, value in sub_dict.items():
        # Add the key-value pair to the batch
        batch[key] = value

    # Run the write_batch coroutine asynchronously
    asyncio.run(write_batch(batch))

In [76]:
#map ticker to industry (OK)
#get tickers col as list from master news dataframe (rows) (OK)
#map tickers in list 10 industries boolean col (OK)
#create 10 industry dataframes, 100 rows each, order by time (OK)
#write a python program that run every x mins to get new news and append to the master dataframe, and also append to the 10 industries dataframes.
#user requests for news -> look up pref in firebase by user hashid -> merge industries dataframes -> push to front end 100 rows but limit displace 10 rows at a time

In [121]:
# Create a dictionary to hold the smaller dataframes
industry_data = {}

# Iterate through the industry columns
for col in industry_cols:
    # Get the most recent 100 rows where the industry boolean column is true
    industry_rows = news_data[news_data[col] == True].sort_values(by='published_utc', ascending=False).head(100)
    
    # Add the dataframe to the dictionary using the column name as the key
    industry_data[col] = industry_rows.set_index('id').T.to_dict()

    # Print the number of rows added to the dataframe
    print(f"{col}: {len(industry_rows)} rows added")

/tmp/ipykernel_817/254609390.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  industry_data[col] = industry_rows.set_index('id').T.to_dict()


services: 100 rows added


/tmp/ipykernel_817/254609390.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  industry_data[col] = industry_rows.set_index('id').T.to_dict()
/tmp/ipykernel_817/254609390.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  industry_data[col] = industry_rows.set_index('id').T.to_dict()
/tmp/ipykernel_817/254609390.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  industry_data[col] = industry_rows.set_index('id').T.to_dict()


manufacturing: 100 rows added
construction: 100 rows added
wholesale: 100 rows added


/tmp/ipykernel_817/254609390.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  industry_data[col] = industry_rows.set_index('id').T.to_dict()


transportation: 100 rows added


/tmp/ipykernel_817/254609390.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  industry_data[col] = industry_rows.set_index('id').T.to_dict()


finance: 100 rows added
retail: 100 rows added
public_administration: 0 rows added
mining: 100 rows added
agriculture: 100 rows added


/tmp/ipykernel_817/254609390.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  industry_data[col] = industry_rows.set_index('id').T.to_dict()
/tmp/ipykernel_817/254609390.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  industry_data[col] = industry_rows.set_index('id').T.to_dict()
/tmp/ipykernel_817/254609390.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  industry_data[col] = industry_rows.set_index('id').T.to_dict()


In [122]:
#upload 100 news industries data to db
doc_ref = db.collection('industry_news').document('dict')
doc_ref.set(industry_data)

update_time {
  seconds: 1679437164
  nanos: 283412000
}

user data feed

In [84]:
def get_ticker_vector(user_pref, ticker_map_dict, ticker_sic_map):
    ticker_vector = [0] * len(ticker_map_dict)
    
    for ticker, industry in ticker_sic_map.items():
        if industry is None:
            continue
        if industry in user_pref and user_pref[industry]:
            ticker_vector[ticker_map_dict[ticker]] += 1
        elif industry in user_pref and not user_pref[industry]:
            ticker_vector[ticker_map_dict[ticker]] -= 1
    
    return ticker_vector

In [88]:
# Get all documents from the user_preference collection
user_pref_docs = db.collection('user_preference').stream()

# Initialize an empty dictionary to store user preferences for each ticker
user_ticker_pref = {}

# Iterate through each user preference document
for doc in user_pref_docs:
    # Get the user preference dictionary from the document data
    user_pref = doc.to_dict()
    
    # Initialize a list to store the ticker vector for this user
    ticker_vector = [0] * len(ticker_map_dict)
    
    # Iterate through each ticker in the ticker_sic_map
    for ticker, industry in ticker_sic_map.items():
        # Get the index of this ticker in the ticker_vector using the ticker_map_dict
        ticker_index = ticker_map_dict[ticker]
        
        # Check if the industry for this ticker matches any industry in the user preference
        for pref_industry, pref_value in user_pref.items():
            if industry == pref_industry and pref_value:
                # Increase the value of this ticker's index in the ticker_vector
                ticker_vector[ticker_index] += 1
            elif industry == pref_industry and not pref_value:
                # Decrease the value of this ticker's index in the ticker_vector
                ticker_vector[ticker_index] -= 1
        
    # Add this user's ticker vector to the user_ticker_pref dictionary
    user_ticker_pref[doc.id] = ticker_vector

In [89]:
# Get all documents from the user_preference collection
user_pref_docs = db.collection('user_preference').stream()

# Initialize an empty dictionary to store user preferences for each ticker
user_ticker_pref = {}

# Iterate through each user preference document
for doc in user_pref_docs:
    # Get the user preference dictionary from the document data
    user_pref = doc.to_dict()
    
    # Initialize a list to store the ticker vector for this user
    ticker_vector = [0] * len(ticker_map_dict)
    
    # Iterate through each ticker in the ticker_sic_map
    for ticker, industry in ticker_sic_map.items():
        # Get the index of this ticker in the ticker_vector using the ticker_map_dict
        ticker_index = ticker_map_dict[ticker]
        
        # Check if the industry for this ticker matches any industry in the user preference
        for pref_industry, pref_value in user_pref.items():
            if industry == pref_industry and pref_value:
                # Increase the value of this ticker's index in the ticker_vector
                ticker_vector[ticker_index] += 1
            elif industry == pref_industry and not pref_value:
                # Decrease the value of this ticker's index in the ticker_vector
                ticker_vector[ticker_index] -= 1
        
    # Add this user's ticker vector to the user_ticker_pref dictionary
    user_ticker_pref[doc.id] = ticker_vector

In [103]:
# Get all documents from the user_preference collection
user_pref_docs = db.collection('user_preferences').stream()

# Initialize an empty dictionary to store user preferences for each ticker
user_ticker_pref = {}

# Iterate through each user preference document
for doc in user_pref_docs:
    # Get the user preference dictionary from the document data
    user_pref = doc.to_dict()
    
    # Initialize a list to store the ticker vector for this user
    ticker_vector = [0] * len(ticker_map_dict)
    
    # Iterate through each ticker in the ticker_sic_map
    for ticker, industry in ticker_sic_map.items():
        if ticker=="nan":
            pass
        else:
            # Get the index of this ticker in the ticker_vector using the ticker_map_dict
            if ticker not in ticker_map_dict.keys():
                print(ticker)
            ticker_index = ticker_map_dict[ticker]

            # Check if the industry for this ticker matches any industry in the user preference
            for pref_industry, pref_value in user_pref.items():
                if industry == pref_industry and pref_value:
                    # Increase the value of this ticker's index in the ticker_vector
                    ticker_vector[ticker_index] += 1
                elif industry == pref_industry and not pref_value:
                    # Decrease the value of this ticker's index in the ticker_vector
                    ticker_vector[ticker_index] -= 1
        
    # Add this user's ticker vector to the user_ticker_pref dictionary
    user_ticker_pref[doc.id] = ticker_vector

In [105]:
# Iterate through the user_ticker_pref dictionary and set values in the collection
for user_id, ticker_vector in user_ticker_pref.items():
    # Set the document reference in the user_pref_ticker collection using the user_id
    doc_ref = db.collection('user_pref_ticker').document(user_id)

    # Add the ticker_vector to the document
    doc_ref.set({'ticker_vector': ticker_vector})

In [ ]:
news_data["tickers_index"] = news_data.tickers.apply(lambda tickers: [ticker_map_dict.get(ticker, None) for ticker in tickers])

In [112]:
user = user_ticker_pref["2qbeT9d3aCfrSk0PLbsJO10mrV73"]

In [114]:
user_pref_news = list(map(lambda x: np.nanmean([user[i] for i in x if i is not None]), news_data['tickers_index']))

In [123]:
user_pref_docs = db.collection('user_preferences').stream()
industry_news_ref = db.collection('industry_news').document("dict")
industry_news = industry_news_ref.get().to_dict()
print(industry_news.keys())
for doc in user_pref_docs:
    # Get the user preference dictionary from the document data
    user_pref = doc.to_dict()

    # Extract the industry names for which the value is True
    industry_names = [key for key, value in user_pref.items() if value]
    print(industry_names)

    # Create an empty dictionary to store the merged documents
    merged_docs = {}

    # Loop through each industry name and get the corresponding document from Firestore
    for name in industry_names:
        merged_docs.update(industry_news[name])

    # Sort the merged documents by publish_utc in descending order
    sorted_docs = dict(sorted(merged_docs.items(), key=lambda x: x[1]['publish_utc'], reverse=True))

    # Get the 100 most recent rows from the sorted documents
    most_recent_docs = dict(list(sorted_docs.items())[:5])
    
    print(most_recent_docs)


dict_keys(['construction', 'agriculture', 'transportation', 'manufacturing', 'wholesale', 'public_administration', 'mining', 'finance', 'retail', 'services'])
['manufacuring', 'construction', 'transportation', 'algriculture', 'public_administration', 'mining', 'wholesale', 'finance', 'retail', 'services']


KeyError: 'manufacuring'